In [1]:
pip install requests


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests

In [4]:
api_key = 'b6a35b22-ac88-4200-a4ed-f50c0c21e159'
headers = {
    'Authorization': f'Bearer {api_key}'
}


In [ ]:
url = 'https://api.elections.kalshi.com/trade-api/v2/markets'
response = requests.get(url, headers=headers)


ConnectionError: HTTPSConnectionPool(host='trading-api.kalshi.com', port=443): Max retries exceeded with url: /v1/markets (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x107b35950>: Failed to resolve 'trading-api.kalshi.com' ([Errno 8] nodename nor servname provided, or not known)"))

In [8]:
if response.status_code == 200:
    markets = response.json().get('markets', [])
    sports_markets = [market for market in markets if 'sports' in market.get('categories', [])]

    # Write output to a file
    with open("kalshi_sports_markets.txt", "w") as file:
        for market in sports_markets:
            file.write(f"Market Name: {market.get('title')}\n")
            file.write(f"Event Date: {market.get('end_date')}\n")
            file.write(f"Yes Price: {market.get('yes_price')}\n")
            file.write(f"No Price: {market.get('no_price')}\n")
            file.write('-' * 20 + '\n')

    print("Sports market data saved to 'kalshi_sports_markets.txt'.")
else:
    print(f"Failed to retrieve markets: {response.status_code}")

Sports market data saved to 'kalshi_sports_markets.txt'.
